In [1]:
%%file InputParser.py
import xml.etree.ElementTree as ET
import numpy as np
from copy import deepcopy

class InputParser:

    """
    This class Input Parser takes the input of a xml file
    and creates useful variables can be called by other functions or users


    INPUTS
    =======
    file_name: the xml filename.

    RETURNS
    ========
    ###### After init, following class variables can be used:
    'equation' - an equation of the chemical reaction to solve
    'id'- the number of reactions in the file
    'products'- the output of the chemical equation
    'rateCoeffParams'- the variables needed to calculate k, or k
    'reactants' - the input of the chemical equation
    'reversible'- yes/no if a reversable equation
    'type'- the type of reaction (i.e. 'elementary')

    EXAMPLES
    =========
    >>> input_ = InputParser('rxns.xml')
    Finished reading xml input file
    >>> print(input_.species)
    ['H', 'O', 'OH', 'H2', 'H2O', 'O2']
    """


    def __init__(self, file_name):
        """
        INPUT
        =====
        file_name: string
                   name of xml input file

        """
        self.file_name = file_name
        self.raw = ET.parse(self.file_name).getroot()
        self.species = self.get_species()
        self.reactions = self.get_reactions()
        self.nu_react, self.nu_prod = self.get_nu()
        self.rate_coeff_params = self.get_rate_coeff_params()
        print("Finished reading xml input file")

    def get_species(self):
        """
        Retrieves species array from xml file
        """
        try:
            species = self.raw.find('phase').find('speciesArray').text.strip().split()
        except AttributeError:
            print('ERROR: Check that valid species array is provided in xml input file')
            raise
        if len(species)==0:
            raise ValueError('ERROR: Species array needs to be provided')
        else:
            return species


    def get_reactions(self):
        """
        Returns list of dictionaries of information about each reaction in the xml file
        """
        def parse_rate_coeff(reaction, reaction_dict):
            rc_ = reaction.findall('rateCoeff')
            if len(rc_) > 1:
                raise NotImplementedError('ERROR: Only irreversible reactions are currently supported')
            elif len(rc_)==0:
                raise ValueError('Rate coefficient data appear to be missing')
            rc_ = rc_[0]
            reaction_dict['rateCoeffParams'] = dict()
            if rc_.find('Constant') is not None:
                reaction_dict['rateCoeffParams']['type'] = 'Constant'
                try:
                    k = float(rc_.find('Constant').find('k').text)

                except AttributeError:
                    print('ERROR: Value of k must be provided for constant rate coefficient')
                    raise
                if k<=0:
                    raise ValueError('k must be non-negative')
                else:
                    reaction_dict['rateCoeffParams']['k'] = k

            elif rc_.find('Arrhenius') is not None:
                reaction_dict['rateCoeffParams']['type'] = 'Arrhenius'
                try:
                    A = float(rc_.find('Arrhenius').find('A').text)
                    E = float(rc_.find('Arrhenius').find('E').text)
                except AttributeError:
                    print('ERROR: Values of A and E must be provided for Arrhenius rate coefficient')
                    raise
                if A <=0 or E<0:
                    raise ValueError('A must always be positive and E must be non-negative')
                else:
                    reaction_dict['rateCoeffParams']['A'] = A
                    reaction_dict['rateCoeffParams']['E'] = E

            elif rc_.find('modifiedArrhenius') is not None:
                reaction_dict['rateCoeffParams']['type'] = 'modifiedArrhenius'
                try:
                    A= float(rc_.find('modifiedArrhenius').find('A').text)
                    b = float(rc_.find('modifiedArrhenius').find('b').text)
                    E = float(rc_.find('modifiedArrhenius').find('E').text)
                except AttributeError:
                    print('ERROR: Values of A, b, and E must be provided for modified Arrhenius rate coefficient')
                    raise
                if A <=0 or E<0 or np.iscomplex(b):
                    raise ValueError('ERROR: A must always be positive, E must be non-negative, and b must be real')
                else:
                    reaction_dict['rateCoeffParams']['A'] = A
                    reaction_dict['rateCoeffParams']['b'] = b
                    reaction_dict['rateCoeffParams']['E'] = E
            else:
                raise NotImplementedError('This type of reaction rate coefficient has not been implemented. Current supported types are constant, Arrhenius, and modified Arrhenius.')

        reactions = []
        #check that only one system of reactions is present in the file
        rxndata = self.raw.findall('reactionData')
        if len(rxndata) > 1:
            raise ValueError('ERROR: Only one reaction system allowed in input file')
        elif len(rxndata)==0 or len(rxndata[0])==0:
            raise ValueError('Error: No reactions appear to be present in input file')

        else:
            for i, reaction in enumerate(rxndata[0]):
                reactions.append(deepcopy(reaction.attrib))
                parse_rate_coeff(reaction, reactions[i])
                try:
                    reactions[i]['equation'] = reaction.find('equation').text
                    reactions[i]['reactants'] = {s.split(':')[0]:int(s.split(':')[1]) \
                                         for s in reaction.find('reactants').text.split()}
                    reactions[i]['products'] = {s.split(':')[0]:int(s.split(':')[1]) \
                                         for s in reaction.find('products').text.split()}

                except AttributeError:
                    print("Check that equation, reactants, and products information are present for every reaction")
                    raise

                except ValueError:
                    print("Stoichiometric coefficients must be integers")
                    raise
        return reactions

    def get_nu(self):
        """
        Return tuple of arrays corresponding to stoichiometric coefficients for reactants and for products
        """

        nu_react = np.zeros((len(self.species), len(self.reactions)))
        nu_prod = np.zeros((len(self.species), len(self.reactions)))

        for i, reaction in enumerate(self.reactions):
            if not (reaction['reversible'] == 'no' and reaction['type'] == 'Elementary'):
                raise NotImplementedError('Reactions that are reversible or non-elementary are not yet handled by this program.')
            for specie, stoi in reaction['reactants'].items():
                nu_react[self.species.index(specie), i] = stoi
            for specie, stoi in reaction['products'].items():
                nu_prod[self.species.index(specie), i] = stoi

        return nu_react, nu_prod

    def get_rate_coeff_params(self):
        """getter for rate coefficients"""
        return [reaction['rateCoeffParams'] for reaction in self.reactions]

    def __repr__(self):
        """Return a printable representation of the object."""
        return 'InputParser(file_name=\'{}\')'.format(self.file_name)

    def __len__(self):
        """Return the number of chemical reactions."""
        return len(self.reactions)

Overwriting InputParser.py


In [2]:
from InputParser import *
input_ = InputParser('rxns.xml')
input_.reactions

Finished reading xml input file


[{'equation': 'H + O2 =] OH + O',
  'id': 'reaction01',
  'products': {'O': 1, 'OH': 1},
  'rateCoeffParams': {'A': 35200000000.0, 'E': 71400.0, 'type': 'Arrhenius'},
  'reactants': {'H': 1, 'O2': 1},
  'reversible': 'no',
  'type': 'Elementary'},
 {'equation': 'H2 + O =] OH + H',
  'id': 'reaction02',
  'products': {'H': 1, 'OH': 1},
  'rateCoeffParams': {'A': 0.0506,
   'E': 26300.0,
   'b': 2.7,
   'type': 'modifiedArrhenius'},
  'reactants': {'H2': 1, 'O': 1},
  'reversible': 'no',
  'type': 'Elementary'},
 {'equation': 'H2 + OH =] H2O + H',
  'id': 'reaction03',
  'products': {'H': 1, 'H2O': 1},
  'rateCoeffParams': {'k': 1000.0, 'type': 'Constant'},
  'reactants': {'H2': 1, 'OH': 1},
  'reversible': 'no',
  'type': 'Elementary'}]

In [3]:
print(input_.nu_react)
print(input_.nu_prod)

[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  1.  1.]
 [ 0.  0.  0.]
 [ 1.  0.  0.]]
[[ 0.  1.  1.]
 [ 1.  0.  0.]
 [ 1.  1.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  0.]]


In [4]:
input_.rate_coeff_params

[{'A': 35200000000.0, 'E': 71400.0, 'type': 'Arrhenius'},
 {'A': 0.0506, 'E': 26300.0, 'b': 2.7, 'type': 'modifiedArrhenius'},
 {'k': 1000.0, 'type': 'Constant'}]

In [5]:
print(repr(input_))
print(len(input_))

InputParser(file_name='rxns.xml')
3


In [6]:
input_.species

['H', 'O', 'OH', 'H2', 'H2O', 'O2']